In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import chi2_contingency
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
from sklearn.linear_model import LassoCV
from sklearn.naive_bayes import CategoricalNB
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, make_scorer

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')

In [3]:
train.set_index('Claim Identifier', inplace=True)
train.drop_duplicates(inplace=True)
train.drop('OIICS Nature of Injury Description', axis=1, inplace=True)

train.drop(columns=['Birth Year', 'Age at Injury', 'Number of Dependents', 'WCIO Cause of Injury Code'], inplace=True)

In [4]:
train_num = train.select_dtypes(include=np.number).columns.tolist()
train_cat = train.select_dtypes(exclude=np.number).columns.tolist()

# Numerical columns: Impute with mean
num_imputer = SimpleImputer(strategy="mean")
train[train_num] = pd.DataFrame(
    num_imputer.fit_transform(train[train_num]),
    columns=train_num,
    index=train.index
)

# Categorical columns: Impute with most frequent (mode)
cat_imputer = SimpleImputer(strategy="most_frequent")
train[train_cat] = pd.DataFrame(
    cat_imputer.fit_transform(train[train_cat]),
    columns=train_cat,
    index=train.index
)

In [5]:
X = train.drop('Claim Injury Type', axis=1)
y = train['Claim Injury Type']

X_train, X_val, y_train, y_val = train_test_split(X,y, test_size = 0.3, 
                                                  random_state = 0, 
                                                  stratify = y, 
                                                  shuffle = True)

In [6]:
X_train_num = X_train.select_dtypes(include=np.number).set_index(X_train.index)
X_train_cat = X_train.select_dtypes(exclude=np.number).set_index(X_train.index)

X_val_num = X_val.select_dtypes(include=np.number).set_index(X_val.index)
X_val_cat = X_val.select_dtypes(exclude=np.number).set_index(X_val.index)

In [7]:
scaler = MinMaxScaler()
scaler.fit(X_train_num) #fit to training data
X_train_num_scaled = scaler.transform(X_train_num) # this will return an array
X_train_num_scaled = pd.DataFrame(X_train_num_scaled, columns = X_train_num.columns).set_index(X_train.index) # Convert the array to a pandas dataframe

X_val_num_scaled = scaler.transform(X_val_num)
X_val_num_scaled = pd.DataFrame(X_val_num_scaled, columns = X_val_num.columns).set_index(X_val.index)

In [8]:
label_encoders = {}
for col in X_train_cat.columns:
    le = LabelEncoder()
    combined_data = pd.concat([X_train_cat[col], X_val_cat[col]], axis=0).astype(str)
    le.fit(combined_data)
    X_train_cat[col] = le.transform(X_train_cat[col].astype(str))
    X_val_cat[col] = le.transform(X_val_cat[col].astype(str))
    label_encoders[col] = le

ordinal_encoder = OrdinalEncoder()
y_train_encoded = ordinal_encoder.fit_transform(y_train.values.reshape(-1, 1))
y_val_encoded = ordinal_encoder.transform(y_val.values.reshape(-1, 1))

In [9]:
X_train_combined = pd.concat([X_train_num_scaled, X_train_cat], axis=1)
X_val_combined = pd.concat([X_val_num_scaled, X_val_cat], axis=1)

In [10]:
def model_evaluation(model, X_train, y_train, X_val, y_val):
    print("Model: ", model)
    model.fit(X_train, y_train)
    pred_train = model.predict(X_train)
    pred_val = model.predict(X_val)
    print(f1_score(y_train, pred_train, average='macro'))
    print(f1_score(y_val, pred_val, average='macro'))

In [ ]:
dt = DecisionTreeClassifier()
log = LogisticRegression()

models = [dt, log]
for model in models:
    model_evaluation(model, X_train_combined, y_train, X_val_combined, y_val)

Model:  DecisionTreeClassifier()
0.9999776166190302
0.3862510438112367
Model:  LogisticRegression()
0.10137717656137597
0.10144706369916788
Model:  LassoCV()


ValueError: could not convert string to float: '3. MED ONLY'